### This notebook can be used to edit the mask (or ground truth segmentation) image stacks.

Example: In case of axon segmentation, the masks needed to be eroded. Also, the masks of membranes were created which may be used as an alternative way to train the CNN to detech membranes instead of axon bodies (or cytoplasm).


# **0. Getting started with this notebook**
---
This notebook contains two types of cell: text and code.

*   **Text** cells are formatted using a simple markup language called Markdown. They can be created by clickin on  ``` + text ``` 
*   **Code** cells contain either explanatory text or executable code and its output. They can be created by clickin on  ``` + code ```  

To execute or run the **Code** cell, click on the play button on the left side of the cell.<br>
Double clicking on any cell will toggle the edit and display modes.

For more information about these type of cells you can click [here](https://colab.research.google.com/notebooks/basic_features_overview.ipynb). 

# **1. Getting Set Up**
---

In [ ]:
#@markdown ## Mount the Google drive
#@markdown Once mounted, your files can be browsed on the left hand side panel.

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown ##Set up the IPython notebook

from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Show/Hide code'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)
hide_toggle()

In [ ]:
#@markdown ##Import packages

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import wget

# Installing 
!zenodo_get '10.5281/zenodo.7635510' -w 'urls.txt'
!pip install  patool

hide_toggle()

In [ ]:
#@markdown ## Download dataset
PATHFILE = "/content/drive/MyDrive/chiaraZurzoloLab/CellWalker_iPythonNotebooks/for_CellWalker_paper/Flavia_axon_segmentation/parallel_fiber_segmentation"
PATHTODIRECTORY = "/content/drive/MyDrive/chiaraZurzoloLab/CellWalker_iPythonNotebooks/for_CellWalker_paper/Flavia_axon_segmentation/"
data_path = 'urls.txt'
with open(data_path, 'r') as f:
    URL = f.readline()

print('Downloading data from:')
print(URL)

# Download and unzipping
!wget -O "$PATHFILE" "$URL"
!unzip "$PATHFILE" -d "$PATHTODIRECTORY"

# **2. Input image/mask directories**
---

### Set path to training images
A set of training images along with corresponding segmentation masks are used to train the model to determine segmentation masks for new images. Both images_dir and masks_dir should contain same number of corresponding images.


In [ ]:
# ------------- Initial user input ------------
images_dir = "/content/drive/MyDrive/chiaraZurzoloLab/CellWalker_iPythonNotebooks/for_CellWalker_paper/Flavia_axon_segmentation/em_test" #@param {type:"string"}
masks_dir = "/content/drive/MyDrive/chiaraZurzoloLab/CellWalker_iPythonNotebooks/for_CellWalker_paper/Flavia_axon_segmentation/seg_test"#@param {type:"string"}
hide_toggle()

In [ ]:
#@markdown Create sorted file lists from the input image and mask folders<br>

images_filelist = sorted(os.listdir(images_dir))
masks_filelist = sorted(os.listdir(masks_dir))

print("Number of images:", len(images_filelist))
print("Number of masks:", len(masks_filelist))

assert(len(images_filelist) == len(masks_filelist))

hide_toggle()

Get eroded mask of cells

In [ ]:
def get_eroded_mask(filename):
  # Read mask image
  mask = cv2.imread(filename)[:,:,0] # Assuming that the image has multiple channels
  
  # Get edges
  edges = cv2.Canny(mask, 0, 1)

  # Dilate edges
  kernel = np.ones((3, 3), np.uint8)
  edges = cv2.dilate(edges, kernel, iterations=1)

  # Convert edges to 1,0 array
  edges = edges/255
  #print(np.unique(edges))

  # Get eroded mask
  mask2 = mask*(1-edges)

  return(mask2)

In [ ]:
output_dir = "/content/drive/MyDrive/chiaraZurzoloLab/CellWalker_iPythonNotebooks/for_CellWalker_paper/Flavia_axon_segmentation/seg_test_eroded"

for fname in masks_filelist:
  #print(fname)
  mask2 = get_eroded_mask(masks_dir + "/" + fname)
  #plt.imshow(mask2)
  cv2.imwrite(output_dir + "/" + fname, mask2)

Get membrane mask

In [ ]:
def get_membrane_mask(filename):
  # Read mask image
  mask = cv2.imread(filename)[:,:,0] # Assuming that the image has multiple channels
  
  # Get edges
  edges = cv2.Canny(mask, 0, 1)

  # Dilate edges
  kernel = np.ones((3, 3), np.uint8)
  edges = cv2.dilate(edges, kernel, iterations=1)

  # Convert edges to 1,0 array
  edges = edges/255
  #print(np.unique(edges))

  # Get eroded mask
  mask2 = mask*(1-edges)

  mask2_membrane = mask2 == 0

  return(mask2_membrane*255)

In [ ]:
output_dir = "/content/drive/MyDrive/chiaraZurzoloLab/CellWalker_iPythonNotebooks/for_CellWalker_paper/Flavia_axon_segmentation/seg_test_membrane"

for fname in masks_filelist:
  #print(fname)
  mask2_membrane = get_membrane_mask(masks_dir + "/" + fname)
  #plt.imshow(mask2_membrane)
  cv2.imwrite(output_dir + "/" + fname, mask2_membrane)